# Work FLow
- 검색어, 날짜범위에 해당하는 네이버 뉴스 {언론사, 기사제목, 발행일, url} 크롤링
- 각 url에 접속해서 기사 {내용, 댓글} 파싱
- 기사 제목, 내용을 가지고 {텍스트 분류} => 연예기사 제외
- 월별 단어 출현 빈도수 측정 => 동적 가중치 부여

# Selenium으로 네이버 뉴스 스크랩
- 검색필터: 관련도 순

## version 1. Selenium으로 직접 조작

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Chrome WebDriver 설정
driver = webdriver.Chrome(ChromeDriverManager().install(), options=webdriver.ChromeOptions())

driver.get('https://www.naver.com')

ele = driver.find_element(By.ID, 'query')
ele.send_keys('광진구')
ele.send_keys(Keys.ENTER)

news = driver.find_element(by=By.LINK_TEXT, value='뉴스')
news.click()

# 날짜 필터 걸기
driver.find_element(by=By.LINK_TEXT, value='옵션').click()
driver.find_element(by=By.LINK_TEXT, value='직접입력').click()
## selector scroll 

## version 2. url 값 변경

In [71]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
import pandas as pd 

def main():
    # 검색필터 설정
    keyword = input('검색어를 입력해주세요.')
    start_date = input('시작 날짜를 입력해주세요(예: 2023.01.01)')
    end_date = input('마지막 날짜를 입력해주세요(예: 2023.12.31)')
    
    # url 접속
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=webdriver.ChromeOptions())
    url = f'https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230101to20240319&is_sug_officeid=0&office_category=0&service_area=0'
    driver.get(url)

    # 스크롤
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1)
        new_height = driver.execute_script('return document.body.scrollHeight')
    
        if last_height != new_height:
            last_height = new_height
        else:
            break

    # 기사제목 & url 가져오기 
    titles = []
    urls = []
    pub_dates = []
    press = []

    bs = BeautifulSoup(driver.page_source, 'lxml')
    articles = bs.findAll('div', attrs={'class':'news_area'})
    for article in articles:
        titles.append(article.find('a', attrs={'class':'news_tit'}).text)
        urls.append(article.find('a', attrs={'class':'news_tit'})['href'])
        pub_dates.append(article.find('span', attrs={'class':'info'}).text)
        press.append(article.find('a', attrs={'class':'info press'}).text)

    contents = []
    comments = []
    # # 기사내용 & 댓글 가져오기 
    for url in urls:
        try:
            driver.implicitly_wait(1)
            driver.get(url)
            bs = BeautifulSoup(driver.page_source, 'lxml')
            try:
                contents.append(bs.find('div', attrs={'itemprop':'articleBody'}).text)
            except:
                contents.append('')
        except TimeoutException:
            contents.append('')
            print("페이지 로딩 시간이 초과되었습니다.")
            continue
   
            

    # cvs로 반환
    df = pd.DataFrame({'press':press, 'title':titles, 'pub_date':pub_dates, 'url':urls, 'contents':contents})
    df.to_csv(f'{keyword}_news_{start_date}_{end_date}.csv', index=False, encoding='utf-8-sig')

In [72]:
main()

검색어를 입력해주세요. 광진구
시작 날짜를 입력해주세요(예: 2023.01.01) 2023.01.01
마지막 날짜를 입력해주세요(예: 2023.12.31) 2023.01.01


## version 3. 기사, 제목, 언론사, url 가져와서 csv 반환 

In [90]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select

import requests
import re

import pandas as pd 

def get_urls():
    # 검색필터 설정
    keyword = input('검색어를 입력해주세요.')
    start_date = input('시작 날짜를 입력해주세요(예: 2023.01.01)')
    end_date = input('마지막 날짜를 입력해주세요(예: 2023.12.31)')
    
    # url 접속
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=webdriver.ChromeOptions())
    url = f'https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230101to20240319&is_sug_officeid=0&office_category=0&service_area=0'
    driver.get(url)

    # 스크롤
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1)
        new_height = driver.execute_script('return document.body.scrollHeight')
    
        if last_height != new_height:
            last_height = new_height
        else:
            break

    # 기사제목 & url 가져오기 
    titles = []
    urls = []
    pub_dates = []
    press = []

    bs = BeautifulSoup(driver.page_source, 'lxml')
    articles = bs.findAll('div', attrs={'class':'news_area'})
    for article in articles:
        titles.append(article.find('a', attrs={'class':'news_tit'}).text)
        urls.append(article.find('a', attrs={'class':'news_tit'})['href'])
        # span 태그 중 class=info인 태그가 두 개 존재하는 기사 존재: 두번째 태그가 날짜 값
        info = article.findAll('span', attrs={'class':'info'})
        pub_dates.append(info[-1].text)
        press.append(article.find('a', attrs={'class':'info press'}).text)
            

    # cvs로 반환
    df = pd.DataFrame({'press':press, 'title':titles, 'pub_date':pub_dates, 'url':urls})
    df = df.drop_duplicates(df.columns)
    df.to_csv(f'{keyword}_news_{start_date}_{end_date}.csv', index=False, encoding='utf-8-sig')

In [94]:
get_urls()

검색어를 입력해주세요. 광진구
시작 날짜를 입력해주세요(예: 2023.01.01) 2023.01.01
마지막 날짜를 입력해주세요(예: 2023.12.31) 2023.01.31


## version 3. 언론사 분류 필터링해서 가져오기
- 일간지(1), 방송/통신(2), 경제/IT(3), 지역지(6)

In [104]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select

import requests
import re

import pandas as pd 

def get_urls():
    # 검색필터 설정
    keyword = input('검색어를 입력해주세요.')
    start_date = input('시작 날짜를 입력해주세요(예: 2023.01.01)')
    end_date = input('마지막 날짜를 입력해주세요(예: 2023.12.31)')
    category_map = {"일간지":1, "방송/통신":2, "경제/IT":3, "지역지":6}

    # 리스트 초기화 
    titles = []
    urls = []
    pub_dates = []
    press = []
    categories = []

    for category, category_num in category_map.items():
        # url 접속
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=webdriver.ChromeOptions())
        url = f'https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230101to20240319&is_sug_officeid=0&office_category={category_num}&service_area=0'
        driver.get(url)
    
        # 스크롤
        last_height = driver.execute_script('return document.body.scrollHeight')
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1)
            new_height = driver.execute_script('return document.body.scrollHeight')
        
            if last_height != new_height:
                last_height = new_height
            else:
                break
    
        # 기사제목 & url 가져오기 
        
    
        bs = BeautifulSoup(driver.page_source, 'lxml')
        articles = bs.findAll('div', attrs={'class':'news_area'})
        for article in articles:
            titles.append(article.find('a', attrs={'class':'news_tit'}).text)
            urls.append(article.find('a', attrs={'class':'news_tit'})['href'])
            
            # span 태그 중 class=info인 태그가 두 개 존재하는 기사 존재: 두번째 태그가 날짜 값
            info = article.findAll('span', attrs={'class':'info'})
            pub_dates.append(info[-1].text)
            
            press.append(article.find('a', attrs={'class':'info press'}).text)

            categories.append(category)
        
        
    # cvs로 반환
    df = pd.DataFrame({'category':categories, 'press':press, 'title':titles, 'pub_date':pub_dates, 'url':urls})
    df = df.drop_duplicates(df.columns)
    df.to_csv(f'{keyword}_news_{start_date}_{end_date}.csv', index=False, encoding='utf-8-sig')

In [113]:
get_urls()

검색어를 입력해주세요. 광진구
시작 날짜를 입력해주세요(예: 2023.01.01) 2023.10.01
마지막 날짜를 입력해주세요(예: 2023.12.31) 2023.12.31


# test code

In [107]:
df = pd.read_csv('광진구_news_2024.03.20_2024.03.20.csv')

In [108]:
df = df.drop_duplicates(df.columns)

In [109]:
df.sort_values('pub_date')

,category,press,title,pub_date,url
10,방송/통신,연합뉴스,"조재호·김가영, 프로당구 PBA 시상식 대상 영예",10시간 전,https://www.yna.co.kr/view/AKR2024032002170000...
35,경제/IT,더벨,"[더벨]대한토지신탁, 최초·최다 '싹쓸이'…노하우 '독보적'",11시간 전,https://www.thebell.co.kr/free/content/Article...
20,경제/IT,파이낸셜뉴스언론사 선정,"강남도 아닌데 한번에 8억 급등 '신고가'...집값, 지금이 바닥?",13시간 전,http://www.fnnews.com/news/202403191538587912
7,일간지,국민일보,공동주택 세워 목사·교인 26가구 함께 살아요… ‘거주+신앙공동체’ 특별한...,15시간 전,https://www.kmib.co.kr/article/view.asp?arcid=...
32,경제/IT,매일경제,‘왕중왕전-개인투어-팀리그’ 3관왕 조재호 김가영 나란히 프로당구 대상,17시간 전,https://www.mk.co.kr/article/10968864
0,일간지,서울신문,"광진구, 513억원 규모 ‘광진형 특별융자’ 시행",1시간 전,https://www.seoul.co.kr/news/publicnews/local_...
1,일간지,신아일보,"광진구, 513억 규모 ‘광진형 특별융자’ 시행",1시간 전,http://www.shinailbo.co.kr/news/articleView.ht...
4,일간지,국민일보,용접기술훈련원은 왜?…10년만에 열린 신학교 취업박람회 가보니,2시간 전,https://www.kmib.co.kr/article/view.asp?arcid=...
3,일간지,신아일보,"광진구, '문화예술 진흥' 올해 첫 문화예술위원회 개최",2시간 전,http://www.shinailbo.co.kr/news/articleView.ht...
11,방송/통신,뉴시스언론사 선정,"이해찬 ""추미애, 얼굴이 하도 곱게 생겨서 판사 같지 않았다""",2시간 전,https://www.newsis.com/view/?id=NISX20240320_0...


In [87]:
keyword = '광진구'
start_date = '2023.01.01'
end_date = '2023.01.31'
url = f'https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230101to20240319&is_sug_officeid=0&office_category=0&service_area=0'

url

'https://search.naver.com/search.naver?where=news&query=광진구&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2023.01.01&de=2023.01.31&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230101to20240319&is_sug_officeid=0&office_category=0&service_area=0'

In [103]:
category_map = {"일간지":1, "방송/통신":2, "경제/IT":3, "지역지":6}
for category, index in category_map.items():
    print(index)
    print(category)

1
일간지
2
방송/통신
3
경제/IT
6
지역지


In [ ]:
list = [1, 2, 3]
val = pd.Series